In [1]:
import os
os.chdir("/home/medo/Resume CV extraction/")

In [2]:
#pdf_path = "BSV_VITI_LR_N01_22032022.pdf"
pdf_path = "CV Mehdi Fekih.pdf"

# PDF Miner

In [3]:
import pdfminer

In [4]:
from pdfminer.high_level import extract_pages, extract_text

In [5]:
counter = 0
for single_page_layout in extract_pages(pdf_path):
    counter +=1
    print(f'Page {counter}')
    for elt in single_page_layout:
        print(elt)
    if counter == 1:
        break

Page 1
<LTTextBoxHorizontal(0) 237.824,801.721,358.204,823.721 'Mehdi Fekih\n'>
<LTTextBoxHorizontal(1) 204.424,759.911,391.608,785.161 '8 rue Bernard de Clairvaux, 75003 Paris\n07 82 90 60 71 – mehdi.fekih@edhec.com\n'>
<LTTextBoxHorizontal(2) 28.800,730.234,200.169,742.234 '■ FORMATION ET DIPLÔMES\n'>
<LTTextBoxHorizontal(3) 40.800,704.087,60.794,714.087 '2022\n'>
<LTTextBoxHorizontal(4) 139.800,704.087,335.530,714.087 'Data Scientist, Titre RNCP de niveau 7 (Bac+5)\n'>
<LTTextBoxHorizontal(5) 40.800,680.087,90.782,690.087 '2005 – 2009\n'>
<LTTextBoxHorizontal(6) 139.800,668.087,502.511,690.087 'EDHEC BUSINESS SCHOOL, Master in Management – Campus de Lille\nSpécialisation entrepreneuriat. English Track Program (scolarité entièrement en anglais).\n'>
<LTTextBoxHorizontal(7) 139.800,644.087,462.073,654.087 'Université Paris 7, Licence de Mathématiques Appliquées et Sciences Sociales.\n'>
<LTTextBoxHorizontal(8) 40.800,620.087,90.782,630.087 '2002 – 2004\n'>
<LTTextBoxHorizontal(9) 139.

# pyMuPDF

In [6]:
import fitz
import PIL.Image
import io
import os

In [7]:
pdf = fitz.open(pdf_path)
counter = 1
page_counter = 0
for i in range(len(pdf)):
    page_counter += 1
    page = pdf[i]
    images = page.get_images()
    for image in images:
        base_img = pdf.extract_image(image[0])
        image_data = base_img["image"]
        img = PIL.Image.open(io.BytesIO(image_data))
        extension = base_img["ext"]
        folder = f'extracted_images/page_{page_counter}'
        if not os.path.exists(folder):
            os.makedirs(folder)
        img.save(open(f'{folder}/image{counter}.{extension}', "wb"))
        counter += 1
    if counter ==1:
        break

In [8]:
doc = fitz.open(pdf_path)

count = 0
for i, page in enumerate(doc):
    ext = page.get_text("dict")["blocks"]
    for elt in ext:
        if elt["type"] != 1:
            count += 1
            if count > 2:
                break
            display(elt)

{'number': 0,
 'type': 0,
 'bbox': (237.82423400878906,
  15.915508270263672,
  358.20355224609375,
  40.27878952026367),
 'lines': [{'spans': [{'size': 22.0,
     'flags': 20,
     'font': 'TimesNewRomanPS-BoldMT',
     'color': 5475540,
     'ascender': 0.89111328125,
     'descender': -0.21630859375,
     'text': 'Mehdi Fekih',
     'origin': (237.82423400878906, 35.52000045776367),
     'bbox': (237.82423400878906,
      15.915508270263672,
      358.20355224609375,
      40.27878952026367)}],
   'wmode': 0,
   'dir': (1.0, 0.0),
   'bbox': (237.82423400878906,
    15.915508270263672,
    358.20355224609375,
    40.27878952026367)}]}

{'number': 1,
 'type': 0,
 'bbox': (204.42355346679688,
  55.657752990722656,
  391.6080322265625,
  82.08938598632812),
 'lines': [{'spans': [{'size': 11.0,
     'flags': 4,
     'font': 'TimesNewRomanPSMT',
     'color': 0,
     'ascender': 0.89111328125,
     'descender': -0.21630859375,
     'text': '8 rue Bernard de Clairvaux, 75003 Paris',
     'origin': (209.33460998535156, 65.45999908447266),
     'bbox': (209.33460998535156,
      55.657752990722656,
      386.69403076171875,
      67.83939361572266)}],
   'wmode': 0,
   'dir': (1.0, 0.0),
   'bbox': (209.33460998535156,
    55.657752990722656,
    386.69403076171875,
    67.83939361572266)},
  {'spans': [{'size': 11.0,
     'flags': 4,
     'font': 'TimesNewRomanPSMT',
     'color': 0,
     'ascender': 0.89111328125,
     'descender': -0.21630859375,
     'text': '07 82 90 60 71 – mehdi.fekih@edhec.com',
     'origin': (204.42355346679688, 79.70999145507812),
     'bbox': (204.42355346679688,
      69.90774536132812,
      39

In [9]:
import bigtree

In [10]:
from bigtree import shift_nodes, Node, yield_tree, findall
from bigtree import Node, preorder_iter, postorder_iter, levelorder_iter, levelordergroup_iter

In [11]:
from pandas import DataFrame
from typing_extensions import Self
import os
from os.path import basename
import re
import logging
from typing import Iterable, Sequence


# formatting-related constants for pretty-printing
SHORT_FORMAT_SPEC = "short"
FULL_FORMAT_SPEC = "full"
FILTERED_METADATA_KEYS = ["font", "size", "width", "height", "format", "color"]

#: label to classify nodes
NODE_TYPE_LABEL = "node_type"
NODE_TYPE_DOCUMENT = "DOCUMENT"
NODE_TYPE_PAGE = "PAGE"
NODE_TYPE_LINE = "LINE"
NODE_TYPE_SPAN = "SPAN"
NODE_TYPE_IMAGE = "IMAGE"
NODE_TYPE_TABLE = "TABLE"

class PdfContent(Node):
    """
    Holds content extracted from a PDF file as a tree structure.
    """

    def __init__(self, name: str,
                 text: str = None,
                 image: bytearray = None,
                 table: DataFrame = None,
                 metadata: dict = None,
                 labels: dict = None,
                 parent: Self = None):
        """
        Builds a new PdfContent based on either text or an image.

        :param name: (inherited) a unique label for the content node
        :param parent: (inherited) a parent PdfContent, if not the whole PDF file itself (i.e. the tree's root)
        :param text: some text read from the PDF
        :param image: an image extracted from the PDF
        :param metadata: a dict of misc metadata about the text/image source
        :param labels: a dict of metadata about the text/image with semantic value for processing (e.g. classifiers)
        """

        super().__init__(name, parent=parent, text=text, image=image, table=table, labels=labels, metadata=metadata)

    def is_typed(self, node_type: str) -> bool:
        return self.labels[NODE_TYPE_LABEL] == node_type

    def __format__(self, format_spec) -> str:
        """
        Print format of this PdfContent.
        The "full" format_spec displays all metadata. The "short" one displays no metadata at all.

        :return: a printable representation of the PDF
        """

        def printable_text(n: PdfContent) -> str:
            if n.is_typed(NODE_TYPE_LINE) or n.is_typed(NODE_TYPE_TABLE):
                return n.text
            if n.is_typed(NODE_TYPE_IMAGE) and format_spec == FULL_FORMAT_SPEC:
                return to_ascii(n.image)
            if format_spec != SHORT_FORMAT_SPEC:
                return n.text
            return ''

        def printable_metadata(n: PdfContent) -> str:
            if n.metadata is None or format_spec == SHORT_FORMAT_SPEC:
                return ''
            if format_spec == FULL_FORMAT_SPEC:
                return "{}".format(n.metadata)
            return "{}".format({k: v for k, v in n.metadata.items() if k in FILTERED_METADATA_KEYS})

        return "\n".join([
            "{}{}[{}] {} {}".format(branch, stem, node.node_name, printable_text(node), printable_metadata(node))
            for branch, stem, node in yield_tree(self, style="const")
            if format_spec != SHORT_FORMAT_SPEC or not node.is_typed(NODE_TYPE_SPAN)
        ])

    def find_nodes(self, *node_types) -> list[Self]:
        """
        Returns all sub-nodes of the given type(s).
        :return: a tuple of PdfContents
        """
        return list(findall(self, condition=lambda node: node.labels[NODE_TYPE_LABEL] in node_types))



In [12]:
def read_pdf(file_path) -> PdfContent:
    """
    Reads a source PDF file as a PdfContent.
    The returned data structure is pre-processed: its content is as clean as possible to ease actual processing.

    :param file_path: source PDF path on the file system
    """

    with fitz.open(file_path) as doc:
        root_node = PdfContent(name=basename(file_path),
                               labels={NODE_TYPE_LABEL: NODE_TYPE_DOCUMENT},
                               metadata=doc.metadata)
        for page in doc:
            blocks_list = page.get_text("dict", sort=True)["blocks"]
            page_node = PdfContent(name=f"page-{page.number}",
                                   parent=root_node,
                                   metadata={"page_idx": page.number},
                                   labels={NODE_TYPE_LABEL: NODE_TYPE_PAGE},
                                   text=f"{len(blocks_list)} blocks")
            for block_idx, block in enumerate(blocks_list):
                if "image" in block:
                    image = bytearray(block["image"])
                    image_metadata = {k: v for k, v in block.items() if k != "image"} | {"block_idx": block_idx}
                    PdfContent(name=f"img-{page.number}.{block_idx}",
                               parent=page_node,
                               labels={NODE_TYPE_LABEL: NODE_TYPE_IMAGE},
                               image=image,
                               metadata=image_metadata)
                if "lines" in block:
                    for line_idx, line in enumerate(block["lines"]):
                        spans = [span["text"] for span in line["spans"]]
                        line_metadata = {k: v for k, v in line.items() if k != "spans"} | {"block_idx": block_idx}
                        line_node = PdfContent(name=f"line-{page.number}.{block_idx}.{line_idx}",
                                               parent=page_node,
                                               labels={NODE_TYPE_LABEL: NODE_TYPE_LINE},
                                               text=''.join(spans),
                                               metadata=line_metadata)
                        for span_idx, span in enumerate(line["spans"]):
                            PdfContent(name=f"span-{page.number}.{block_idx}.{line_idx}.{span_idx}",
                                       parent=line_node,
                                       labels={NODE_TYPE_LABEL: NODE_TYPE_SPAN},
                                       text=span["text"],
                                       metadata={k: v for k, v in span.items() if k != "text"})

        sanitize(root_node, doc)
    return root_node

In [13]:
def prune_empty_lines(pdf: PdfContent):
    """
    Modify the given PdfContent in place, removing all empty lines.
    :param pdf: a root PdfContent
    """
    count = 0
    for line in pdf.find_nodes(NODE_TYPE_LINE):
        if re.fullmatch(" *", line.text):
            shift_nodes(pdf, [line.path_name], [None])
            count += 1
    logging.debug(f"removed {count} empty lines")


def prune_spans(pdf: PdfContent):
    """
    Modify the given PdfContent in place, removing all spans.
    :param pdf: a root PdfContent
    """
    for span in pdf.find_nodes(NODE_TYPE_SPAN):
        shift_nodes(pdf, [span.path_name], [None])


def prune_footers(pdf: PdfContent):
    """
    Search for footers in the given PDF and remove them in place.
    Assume each page may finish with a same footer block, differing from at most a page number.
    :param pdf: a multi-pages PDF document
    """
    footers: list[tuple[str, list[PdfContent]]] = []  # tuples with concatenated text content & nodes
    for page in pdf.find_nodes(NODE_TYPE_PAGE):
        last_block_idx = max([line.metadata["block_idx"] for line in page.find_nodes(NODE_TYPE_LINE)])
        last_lines = [line for line in page.find_nodes(NODE_TYPE_LINE) if line.metadata["block_idx"] == last_block_idx]
        footers.append(("\t".join([line.text for line in last_lines]), last_lines))

    for footer in footers:
        # compare with other text contents
        diff_list = [edit_distance(footer[0], f[0]) for f in footers if f[1] != footer[1]]
        # ignore outliers
        max_outliers = min(len(footers) - 1, FOOTER_MAX_OUTLIERS)  # keep 1+ elements
        for i in range(max_outliers):
            diff_list.remove(max(diff_list))
        # remove nodes with same content as others
        if len(diff_list) > 0 and max(diff_list) <= FOOTER_DISTANCE_THRESHOLD:
            logging.debug(f"removed footer at {[f.path_name for f in footer[1]]}: {footer[0]}")
            shift_nodes(pdf, [f.path_name for f in footer[1]], [None] * len(footer[1]))
def infer_line_format(pdf: PdfContent):
    """
    Enrich lines metadata in place, with "font" and "size" attributes, based on children spans.
    The font is the most used one; the size is the biggest one.
    :param pdf: some PdfContent
    """
    for line in pdf.find_nodes(NODE_TYPE_LINE):
        line_spans = line.find_nodes(NODE_TYPE_SPAN)
        line.metadata["font"] = most_used_metadata(line_spans, normalize_font)
        line.metadata["size"] = max([normalize_size(span) for span in line.find_nodes(NODE_TYPE_SPAN)])

def infer_line_format(pdf: PdfContent):
    """
    Enrich lines metadata in place, with "font" and "size" attributes, based on children spans.
    The font is the most used one; the size is the biggest one.
    :param pdf: some PdfContent
    """
    for line in pdf.find_nodes(NODE_TYPE_LINE):
        line_spans = line.find_nodes(NODE_TYPE_SPAN)
        line.metadata["font"] = most_used_metadata(line_spans, normalize_font)
        line.metadata["size"] = max([normalize_size(span) for span in line.find_nodes(NODE_TYPE_SPAN)])


def most_used_metadata(contents: Iterable[PdfContent], metadata_getter):
    """
    Select one of the most used metadata in the given content.
    :param contents: collection of contents to parse
    :param metadata_getter: how to extract the metadata to compare
    :return: the most used value, or None if none was found
    """
    values_length: dict[int | None] = {None: 0}
    for content in contents:
        value = metadata_getter(content)
        text_length = len(content.text)
        values_length.setdefault(value, 0)
        values_length[value] += text_length
    main_font_length = max(values_length.values())
    return [k for k, v in values_length.items() if v == main_font_length][0]


def merge_similar_sibling_lines(pdf: PdfContent):
    """
    Modifies the given PdfContent in place,
    merging sibling lines when coming from the same block or having similar format.
    :param pdf: a PdfContent with content to merge
    """
    all_pages = list(pdf.find_nodes(NODE_TYPE_PAGE))
    for page in all_pages:
        # create a new page container
        new_page = PdfContent(name=f"{page.node_name}-merged",
                              parent=pdf,
                              labels={NODE_TYPE_LABEL: NODE_TYPE_PAGE},
                              metadata=page.metadata)

        # mark mergeable lines (to the right)
        mergeable_lines = [
            child.is_typed(NODE_TYPE_LINE) and child.right_sibling is not None
            and child.right_sibling.is_typed(NODE_TYPE_LINE) and same_format(child, child.right_sibling)
            for child in page.children
        ]

        i = -1
        j = 0
        while j < len(mergeable_lines):
            if not mergeable_lines[j]:  # end of mergeable sub-list
                if i < j - 1:
                    source_lines = page.children[0:j - i]
                    # create an aggregating line in the new page
                    new_line = PdfContent(name=f"{source_lines[0].node_name}-{source_lines[-1].node_name}",
                                          parent=new_page,
                                          text=re.sub(" +", " ", " ".join([line.text for line in source_lines])),
                                          labels={NODE_TYPE_LABEL: NODE_TYPE_LINE},
                                          metadata={
                                              "sources": {s.node_name: s.metadata for s in source_lines},
                                              "font": normalize_font(source_lines[0]),
                                              "size": normalize_size(source_lines[0]),
                                          })
                    # migrate children and remove merged lines
                    for source_line in source_lines:
                        for child in source_line.children:
                            shift_nodes(pdf, [child.path_name], [f"{new_line.path_name}/{child.node_name}"])
                        shift_nodes(pdf, [source_line.path_name], [None])
                else:
                    unchanged_line = page.children[0]
                    shift_nodes(pdf, [unchanged_line.path_name], [f"{new_page.path_name}/{unchanged_line.node_name}"])
                i = j
            j += 1

        # cleanup
        shift_nodes(pdf, [page.path_name], [None])


def infer_titles(pdf: PdfContent):
    """
    Modifies the given PdfContent in place, introducing a line hierarchy based on font size.
    Images are always considered a direct subsection of their predecessor node.
    :param pdf: a whole PdfContent
    """

    # no hierarchy below the main font size
    main_font_size = most_used_metadata(pdf.find_nodes(NODE_TYPE_LINE), normalize_size)

    for image in pdf.find_nodes(NODE_TYPE_IMAGE):
        if image.left_sibling is not None and image.left_sibling.is_typed(NODE_TYPE_LINE):
            shift_nodes(pdf, [image.path_name], [f"{image.left_sibling.path_name}/{image.node_name}"])

    all_lines = [line for line in pdf.find_nodes(NODE_TYPE_LINE, NODE_TYPE_TABLE)]
    for line in all_lines:
        while is_subsection(line, main_font_size):
            shift_nodes(pdf, [line.path_name], [f"{line.left_sibling.path_name}/{line.node_name}"])


def is_subsection(line: PdfContent, main_font_size: float) -> bool:
    """
    Tells if the given line is a subsection of its left sibling.
    :param line: a line
    :param main_font_size: the font size titles should exceed
    :return: if the given line should be a subsection of its left sibling
    """
    sibling = line.left_sibling
    if sibling is None or not sibling.is_typed(NODE_TYPE_LINE) or normalize_size(sibling) <= main_font_size:
        return False
    return normalize_size(line) < normalize_size(sibling)


def normalize_size(content: PdfContent) -> (float, None):
    """
    Normalize the given node's size in order to compare values more leniently.
    :param content: some PdfContent
    :return: a comparable, rounded size; or None if not applicable
    """
    raw_size = content.metadata["size"] if "size" in content.metadata else None
    if raw_size is None:
        return None
    return round(float(raw_size) * 4, 0) / 4  # rounded to 0.25


def normalize_font(content: PdfContent) -> (str, None):
    """
    Normalize the given node's font name in order to compare values more leniently.
    :param content: some PdfContent
    :return: a simpler, comparable font name; or None if not applicable
    """
    raw_font = content.metadata["font"] if "font" in content.metadata else None
    if raw_font is None:
        return None
    return re.sub("-(Bold|Italic|Black|Regular)+|MT\\b|PS\\b", "", raw_font)  # ignore modifiers & suffixes


def same_format(content1: PdfContent, content2: PdfContent) -> bool:
    """
    Returns if the given PdfContents have similar text formatting.
    :param content1: some content
    :param content2: another content
    :return: if the given contents are similar, formatting-wise; or None if not comparable
    """
    font1 = normalize_font(content1)
    font2 = normalize_font(content2)
    size1 = normalize_size(content1)
    size2 = normalize_size(content2)
    if font1 is None or font2 is None or size1 is None or size2 is None:
        return False
    same_font = font1 == font2
    same_font_size = abs(size1 - size2) < .5
    return same_font and same_font_size


def to_ascii(image: bytearray) -> str:
    """
    Provides a textual, terminal-compatible view of the given image.
    For debug purposes only.

    :param image: any image
    :return: a colored ASCII-art image
    """

    temp_file = tempfile.NamedTemporaryFile(prefix="unibsv-", delete=False)
    with temp_file:
        temp_file.write(image)
    ascii_art = AsciiArt.from_image(temp_file.name).to_ascii(columns=100)
    os.unlink(temp_file.name)
    return "\n" + ascii_art

def merge_all_pages(pdf) -> PdfContent:
    """
    Modifies the given PdfContent in place, merging all pages into a new one.
    :param pdf: a root PdfContent with pages
    :return: the new page with all content
    """
    all_pages = [page for page in pdf.find_nodes(NODE_TYPE_PAGE)]

    # add a new page as container
    single_page = PdfContent(name="single-page",
                             parent=pdf,
                             labels={NODE_TYPE_LABEL: NODE_TYPE_PAGE},
                             metadata={"sources": {page.node_name: page.metadata for page in all_pages}})

    # move all children
    for page in all_pages:
        for from_line in page.children:
            shift_nodes(pdf, [from_line.path_name], [f"{single_page.path_name}/{from_line.node_name}"])

    # cleanup
    for page in all_pages:
        shift_nodes(pdf, [page.path_name], [None])

    logging.debug(f"merged {len(all_pages)} pages")
    return single_page

In [14]:
def sanitize(pdf: PdfContent, doc: fitz.Document):
    """
    Process in place the given PdfContent to ensure it's in a convenient format for further processing.
    * Based on font/size metadata, aggregate spans and lines into paragraphs
    * Include tables whenever possible
    * Remove useless lines whenever possible (e.g. footers)
    * Infer a title/section hierarchy based on font size and line order
    * Simplify the tree structure (single page and no spans)
    :param pdf: a raw PdfContent
    :param doc: the source document (must not be closed yet, to enable extracting additional content)
    """

    prune_empty_lines(pdf)
    infer_line_format(pdf)
    prune_spans(pdf)  # at this point spans are not useful anymore
    #prune_footers(pdf)
    #substitute_tables(pdf, doc)
    merge_all_pages(pdf)
    #merge_similar_sibling_lines(pdf)
    infer_titles(pdf)

In [16]:
pdf = read_pdf(pdf_path)

In [17]:
for node in preorder_iter(pdf):
    break
    print(node.text)

# Spacy 

# Gradio

In [30]:
import gradio as gr
import shutil

UPLOAD_DIR = "uploads"  # Directory to save the uploaded files

# Create the upload directory if it doesn't exist
os.makedirs(UPLOAD_DIR, exist_ok=True)

def upload_file(file):
    file_path = os.path.join(UPLOAD_DIR, file.name)
    shutil.move(file_path, UPLOAD_DIR)

with gr.Blocks() as demo:
    file_output = gr.File()
    upload_button = gr.UploadButton("Click to Upload a File", file_types=["pdf", "doc"], file_count="single")
    upload_button.upload(upload_file, upload_button, file_output)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7871
Running on public URL: https://f235a19b7e4f071c38.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


/tmp/gradio/e85f62cb2da7d7af944bfbbda1a7bbdfa3ba3bb0/livraison.png
